In [1]:
import numpy as np
import matplotlib.pyplot as plt
import control as ct
from scipy.linalg import solve_discrete_are

Pendulum Physical Parameters

In [2]:
dt = 0.01 # timestep [s] 

J = 0.00054 # pendulum inertia with respect to CoG
b_r = 0.001  # motor friction
Ke = 0.29 # Motor EMF constant

Rm = 40 # motor coil resistance
Lm = 0.001 # motor induction

Vmax = 24 # maximal voltage

Nonlinear model

In [3]:
# Exercise:
# Build the model using eq. 10

# function that describes nonlinear dynamics
def nonlinear_dynamics(theta, d_theta, i, voltage):
    # Input:
    #   theta - pendulum angle [rad]
    #   d_theta - pendulum angular velocity [rad/s]
    #   i - wheel angle [rad]
    #   voltage - motor input voltage [volt]
    
    _dd_theta = Ke/J * i - b_r/J * d_theta
    _d_i = -Rm/Lm * i - Ke/Lm * d_theta + voltage/Lm

    _theta = theta + d_theta * dt
    _d_theta = d_theta + _dd_theta * dt
    _i = i + _d_i * dt

    return np.array([_theta, _d_theta, _i])

In [4]:
# Model Simulation

t0 = 0 # start time [s] 
t1 = 6 # stop time [s] 

nt = int(t1/dt) + 1 # Number of points of sim time
T = np.linspace(t0, t1, nt)

x0 = np.array([1.57, 0, 0, 0]) #pend angle, pend angular vel, wheel angle, wheel ang vel
u = 0.0 # voltage [V]

X = [x0]
U = [u]

x_cur = x0.copy()

for t in T[:-1]:
    x_next = nonlinear_dynamics(x_cur[0], x_cur[1], x_cur[2], x_cur[3], u)

    X.append(x_next)
    U.append(u)

    x_cur = x_next.copy()

X = np.array(X)
U = np.array(U)
x1 = X[:, 0]
x2 = X[:, 1]
x3 = X[:, 2]
x4 = X[:, 3]


# Visualization 

plt.figure(figsize=(7, 7))

plt.subplot(4, 1, 1) 
plt.plot(T, x1, 'blue') 
plt.grid() 
plt.legend(labels=('theta [rad]',))
plt.xlabel("Время [с]")

plt.subplot(4, 1, 2) 
plt.plot(T, x2, 'blue') 
plt.grid() 
plt.legend(labels=('d_theta [rad/s]',))
plt.xlabel("Время [с]")

plt.subplot(4, 1, 3) 
plt.plot(T, x3, 'blue') 
plt.grid() 
plt.legend(labels=('phi [rad]',))
plt.xlabel("Время [с]")

plt.subplot(4, 1, 4) 
plt.plot(T, x4, 'blue') 
plt.grid() 
plt.legend(labels=('d_phi [rad/s]',))
plt.xlabel("Время [с]")


plt.tight_layout()
plt.show()

# Exercises:
# Question: Why do we see such behaviour?
# Try to play with initial state and control input in order to better understand the behaviour

TypeError: nonlinear_dynamics() takes 4 positional arguments but 5 were given

State Space Model

In [ ]:
# Exercise:
# Create State Space Matrices using eq. (12)

# elements of matrix A and vector B
a22 = -b_r/J
a23 = Ke/J

a32 = -b_r/Lm
a33 = -Rm/Lm

b3 = -1/Lm

# State transition matrix A and control vector B
A = np.array([  [0,   1,   0],
                [0, a22, a23],
                [0, a32, a33]])

B = np.array([  [0],
                [0],
                [b3]])

# Output matrices and vectors
C = np.array([  [1, 0, 0], 
                [0, 1, 0],   
                [0, 0, 1]])    

D = np.array([[0],[0],[0]])

# Creating continuous state space model via Control Toolbox
sys = ct.ss(A, B, C, D)

In [ ]:
# Model Simulation 

t0 = 0 # [s] 
t1 = 6.0 # [s] 

nt = int(t1/dt) + 1 # Number of points of sim time
t = np.linspace(t0, t1, nt)

x0 = np.array([0.01, 0, 0, 0]) # pend angle [rad], pend angular vel [rad/sec], wheel angle [rad], wheel speed [rad/s]
u = 5 * np.ones(nt) # voltage [V]

# t_, yout = ct.forced_response(sys, t, u, x0)
t_, yout = ct.initial_response(sys, t, x0)

# Visualization 

x1 = yout[0, :]
x2 = yout[1, :]
x3 = yout[2, :]
x4 = yout[3, :]

plt.figure(1, figsize=(8, 8))

plt.subplot(4, 1, 1) 
plt.plot(t, x1, 'blue') 
plt.grid() 
plt.legend(labels=('x1 [rad]',))

plt.subplot(4, 1, 2) 
plt.plot(t, x2, 'green') 
plt.grid() 
plt.legend(labels=('x2 [rad/sec]',))

plt.subplot(4, 1, 3) 
plt.plot(t, x3, 'red') 
plt.grid() 
plt.legend(labels=('x3 [rad]',))

plt.subplot(4, 1, 4) 
plt.plot(t, x4, 'black') 
plt.grid() 
plt.legend(labels=('x4 [rad/sec]',))

# Exercises:
# Question: Why does this plot go to infinity?
# Try to play with initial state and control input using forced response

In [ ]:
# Convert to discrete form
sysd = ct.c2d(sys, dt)
Ad = sysd.A
Bd = sysd.B

# Exercises:
# Write your own code that converts model from continuous to discrete time domain using eq. 13

Control

Stabilization in upright position via LQR Control

In [ ]:
# Cost function weight parameters
Q = np.array([  [1, 0, 0, 0],
                [0, 1, 0, 0],
                [0, 0, 1, 0],
                [0, 0, 0, 1]])
R = np.array([1])

# Solve the problem

# simple way
K, sys, E = ct.dlqr(Ad, Bd, Q, R)

# Exercise:
# Для решения задачи LQR использована функция dlqr библиотеки Control Toolbox. В библиотеке scipy есть функция solve_discrete_are, решающая уравнения Риккати.
# Напиши свой собственный код, который бы вычислял матрицу K по уравнению (16), используя библиотечную функцию solve_discrete_are.

In [ ]:
print("K Gains:")
print(K[0])

# Exercise:
# In the previous cell you can see the commented line 'K, sys, E = ct.dlqr(Ad, Bd, Q, R)'. The function dlqr does the same operations as you have done for getting K gains. Uncomment this line and compare the values of K matrix.

Stability Analysis

In [ ]:
# The system is said to be asymptotically stable if all eigenvalues of the closed-loop matrix A_cl = A - BK
# lie strictly inside the unit circle in the complex plane

# ...put your code here...

print("Eigenvalues of closed-loop system:", eigvals)
if np.all(np.abs(eigvals) < 1):
    print("System is asympthotically stable")
else:
    print("System is unstable")

Closed-Loop System Simulation

In [ ]:
# State space model simulation

x0 = np.array([0.01, 0.0, 0.0, 0.0])  # начальное состояние (наклон маятника)
T = np.arange(0, 4, dt)
X = [x0]
U = []

for t in T[:-1]:
    u = -K @ X[-1]
    x_next = Ad @ X[-1] + Bd.flatten() * u
    X.append(x_next)
    U.append(u)

X = np.array(X)
U = np.array(U)
x1 = X[:, 0]
x2 = X[:, 1]
x3 = X[:, 2]
x4 = X[:, 3]

# Visualization

plt.figure(figsize=(10, 10))

plt.subplot(5, 1, 1) 
plt.plot(T, x1, 'blue') 
plt.grid() 
plt.legend(labels=('x1 [rad]',))
plt.xlabel("Время [с]")

plt.subplot(5, 1, 2) 
plt.plot(T, x2, 'blue') 
plt.grid() 
plt.legend(labels=('x2 [rad/s]',))
plt.xlabel("Время [с]")

plt.subplot(5, 1, 3) 
plt.plot(T, x3, 'blue') 
plt.grid() 
plt.legend(labels=('x3 [rad]',))
plt.xlabel("Время [с]")

plt.subplot(5, 1, 4) 
plt.plot(T, x4, 'blue') 
plt.grid() 
plt.legend(labels=('x4 [rad/s]',))
plt.xlabel("Время [с]")

plt.subplot(5, 1, 5) 
plt.plot(T[:-1], U, 'red') 
plt.grid() 
plt.legend(labels=('u [V]',))
plt.xlabel("Время [с]")

plt.tight_layout()
plt.show()

In [ ]:
# Nonlinear model simulation

t0 = 0 # [s] 
t1 = 4.0 # [s] 

nt = int(t1/dt) + 1 # Number of points of sim time
T = np.linspace(t0, t1, nt)

x0 = np.array([0.01, 0.0, 0.0, 0.0]) #pend angle, pend angular vel, wheel speed
u = 0.0 # [V]

X = [x0]
U = [u]

x_cur = x0.copy()

for t in T[:-1]:
    u = -K @ x_cur
    u[0] = np.clip(u[0], -Vmax, Vmax)

    x_next = nonlinear_dynamics(x_cur[0], x_cur[1], x_cur[2], x_cur[3], u[0])

    X.append(x_next)
    U.append(u[0])

    x_cur = x_next.copy()

X = np.array(X)
U = np.array(U)
x1 = X[:, 0]
x2 = X[:, 1]
x3 = X[:, 2]
x4 = X[:, 3]

# Simulated data visualization 

plt.figure(figsize=(10, 10))

plt.subplot(5, 1, 1) 
plt.plot(T, x1, 'blue') 
plt.grid() 
plt.legend(labels=('theta [rad]',))
plt.xlabel("Время [с]")

plt.subplot(5, 1, 2) 
plt.plot(T, x2, 'blue') 
plt.grid() 
plt.legend(labels=('d_theta [rad/s]',))
plt.xlabel("Время [с]")

plt.subplot(5, 1, 3) 
plt.plot(T, x3, 'blue') 
plt.grid() 
plt.legend(labels=('phi [rad]',))
plt.xlabel("Время [с]")

plt.subplot(5, 1, 4) 
plt.plot(T, x4, 'blue') 
plt.grid() 
plt.legend(labels=('d_phi [rad/s]',))
plt.xlabel("Время [с]")

plt.subplot(5, 1, 5) 
plt.plot(T, U, 'red') 
plt.grid() 
plt.legend(labels=('u [V]',))
plt.xlabel("Время [с]")

plt.tight_layout()
plt.show()

# Exercise
# Play with the cost function weights and check how transient responce changes its characteristics. Choose the best values.

Swing-Up Control

In [ ]:
# Nonlinear model simulation
E_ref = (m1 * L1 + m2 * L2) * g
k = 30.0
epsilon = 0.16

t0 = 0 # [s] 
t1 = 4.0 # [s] 

nt = int(t1/dt) + 1 # Number of points of sim time
T = np.linspace(t0, t1, nt)

x0 = np.array([3.14, 0.0, 0.0, 0.0]) #pend angle, pend angular vel, wheel speed
u = 0.0 # [V]
E = 0.5 * (m1 * L1**2 + m2 * L2**2 + I1 + I2) * x_cur[1]**2 + (m1 * L1 + m2 * L2) * g * np.cos(x_cur[0]) # system energy

X = [x0]
U = [u]
E_arr = [E]
E_ref_arr = [E_ref]
t_switch = 0.0

x_cur = x0.copy()

for t in T[:-1]:
    # calculate current system energy
    E = 0.5 * (m1 * L1**2 + m2 * L2**2 + I1 + I2) * x_cur[1]**2 + (m1 * L1 + m2 * L2) * g * np.cos(x_cur[0])
    # check if energy error less than threshold
    if np.abs(E_ref - E) < epsilon:
        # stabilization mode (LQR control)
        u = -K @ x_cur
        u = u[0]
        if t_switch == 0.0:
            t_switch = t
    else:
        # swing-up mode (energy-shaping control)
        u = k * (E_ref - E) * np.sign(-x_cur[1])
    u = np.clip(u, -Vmax, Vmax)

    x_next = nonlinear_dynamics(x_cur[0], x_cur[1], x_cur[2], x_cur[3], u)

    # save data
    X.append(x_next)
    U.append(u)
    E_arr.append(E)
    E_ref_arr.append(E_ref)

    x_cur = x_next.copy()

X = np.array(X)
U = np.array(U)
E_arr = np.array(E_arr)
E_ref_arr = np.array(E_ref_arr)
x1 = X[:, 0]
x2 = X[:, 1]
x3 = X[:, 2]
x4 = X[:, 3]
e_real = E_arr[:]
e_ref = E_ref_arr[:]

# Data visualization 
plt.figure(figsize=(10, 15))

plt.subplot(6, 1, 1) 
plt.plot(T, x1, 'blue') 
plt.plot([t_switch, t_switch], [0.0, 5.0], 'black', linestyle = 'dashed')
plt.grid() 
plt.legend(labels=('theta [rad]',))
plt.xlabel("Время [с]")

plt.subplot(6, 1, 2) 
plt.plot(T, x2, 'blue') 
plt.plot([t_switch, t_switch], [-17.0, 7.0], 'black', linestyle = 'dashed')
plt.grid() 
plt.legend(labels=('d_theta [rad/s]',))
plt.xlabel("Время [с]")

plt.subplot(6, 1, 3) 
plt.plot(T, x3, 'blue') 
plt.plot([t_switch, t_switch], [0.0, 20.0], 'black', linestyle = 'dashed')
plt.grid() 
plt.legend(labels=('phi [rad]',))
plt.xlabel("Время [с]")

plt.subplot(6, 1, 4) 
plt.plot(T, x4, 'blue') 
plt.plot([t_switch, t_switch], [-45.0, 45.0], 'black', linestyle = 'dashed')
plt.grid() 
plt.legend(labels=('d_phi [rad/s]',))
plt.xlabel("Время [с]")

plt.subplot(6, 1, 5) 
plt.plot(T, e_real, 'blue') 
plt.plot(T, e_ref, 'red') 
plt.plot([t_switch, t_switch], [-0.3, 0.3], 'black', linestyle = 'dashed')
plt.grid() 
plt.legend(labels=('e_real []', 'e_ref []'))
plt.xlabel("Время [с]")

plt.subplot(6, 1, 6) 
plt.plot(T, U, 'red') 
plt.plot([t_switch, t_switch], [-12.0, 12.0], 'black', linestyle = 'dashed')
plt.grid() 
plt.legend(labels=('u [V]',))
plt.xlabel("Время [с]")

plt.tight_layout()
plt.show()

# Exercise
# Play with k gain and threshold epsilon. Check how transient responce changes its characteristics and choose the best values.